<a href="https://colab.research.google.com/github/aoifekeane/EE475-Project/blob/main/multi_class_meter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multiclassifier for meter

In [ ]:
seed = 201715226

Random seed for consistency

In [ ]:
lb = { 0: 'Hornpipes',
      1: 'Jigs',
      2: 'Reels', 
      3: 'Polkas',
      4: 'Slides'}

Dictionary to convert label values to actual names

In [ ]:
import glob
import tensorflow as tf
import numpy as np

img_height = 288
img_width = 432

hornpipe = glob.glob('/content/drive/MyDrive/project/images_rgb/hornpipe/*.*')
jig = glob.glob('/content/drive/MyDrive/project/images_rgb/jig/*.*')
polka = glob.glob('/content/drive/MyDrive/project/images_rgb/polka/*.*')
reel = glob.glob('/content/drive/MyDrive/project/images_rgb/reel/*.*')
slide = glob.glob('/content/drive/MyDrive/project/images_rgb/slide/*.*')

data = []
labels = []
count = 0
max = 198
for i in hornpipe:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (img_width,img_height))
    image=np.array(image)
    data.append(image)
    labels.append(0)
    if count >= max:
      break
    count+=1
count = 0
for i in jig:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (img_width,img_height))
    image=np.array(image)
    data.append(image)
    labels.append(1)
    if count >= max:
      break
    count+=1
count = 0
for i in polka:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (img_width,img_height))
    image=np.array(image)
    data.append(image)
    labels.append(2)
    if count >= max:
      break
    count+=1
count = 0
for i in reel:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (img_width,img_height))
    image=np.array(image)
    data.append(image)
    labels.append(3)
    if count >= max:
      break
    count+=1
count = 0
for i in slide:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (img_width,img_height))
    image=np.array(image)
    data.append(image)
    labels.append(4)
    if count >= max:
      break
    count+=1
data = np.array(data)
labels = np.array(labels)


train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=1 - train_ratio, random_state=seed)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=seed) 

*Note, change file paths to where ever the data is stored*
- Reads in image files for max (198) number of files per class
- Splits data into 75:15:10 train, validation, test split

In [ ]:
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
inputs = x_train
targets = y_train
kfold = KFold(n_splits=5, shuffle=True, random_state =seed)
acc_per_fold = []
loss_per_fold = []
fold_no = 1
num_classes = 5

for train, test in kfold.split(inputs, targets):
  model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_width, img_height, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
  ])
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  epochs=10
  history = model.fit(
    inputs[train], targets[train],
    validation_data= (x_val,y_val),
    epochs=epochs
  )
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  fname = '//content/drive/MyDrive/project/multiclass meter models/k folds/meter_model'+str(fold_no)
  model.save(fname)
  
  y_pred = model.predict(inputs[test])
  predicted_categories = tf.argmax(y_pred, axis=1)
  true_categories = tf.concat([targets[test]], axis=0)
  array = confusion_matrix(predicted_categories, true_categories, normalize=None)
  df_cm = pd.DataFrame(array, index = [i for i in ['hornpipe', 'jig', 'polka', 'reel', 'slide']],
                    columns = [i for i in ['hornpipe', 'jig', 'polka', 'reel', 'slide']])
  plt.figure(figsize = (10,7))
  sn.heatmap(df_cm, annot=True)
  print('--------------------')
  print('for fold:', fold_no)
  print(array)
  # Increase fold number
  fold_no = fold_no + 1


- performs 5-fold cross validation on training data
- model is a CNN comprising:
  - 3 2D convolutional layers
  - 3 layers of max pooling
  - 2 instances of 20% dropout
- completes training of model at each fold, then evaluates each model
- displays confusion matrix for each fold
- saves the model at each fold so it can be accessed later


In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/project/multiclass meter models/k folds/meter_model3')

loads in the most effective model

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/project/multiclass meter models/training/model_epoch{epoch:02d}-loss{val_loss:.4f}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

adds in a calllback metric which saves the weights of the model each time validation loss hits a new low

In [ ]:
epochs=70
history = model.fit(
  x = x_train, y = y_train,
  validation_data= (x_val,y_val),
  epochs=epochs, callbacks=[model_checkpoint_callback]
  )

best model is trained on the whole dataset for 70 epochs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')

plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.0])

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.4])
plt.show()

the training and validation accuracy and loss are plotted for the model per epoch

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
y_pred = model.predict(x_test)
predicted_categories = tf.argmax(y_pred, axis=1)
true_categories = tf.concat([y_test], axis=0)
array = confusion_matrix(predicted_categories, true_categories, normalize=None)
df_cm = pd.DataFrame(array, index = [i for i in ['hornpipe', 'jig', 'polka', 'reel', 'slide']],
                  columns = [i for i in ['hornpipe', 'jig', 'polka', 'reel', 'slide']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

confusion matrix plotted for data based on results of test data

In [ ]:
checkpoint_path = '/content/drive/MyDrive/project/multiclass meter models/training/model_epoch53-loss0.0447'
model.load_weights(checkpoint_path)

the weights of the model with minimised validation loss were loaded in to evaluate this

In [ ]:
y_pred = model.predict(x_test)
predicted_categories = tf.argmax(y_pred, axis=1)
true_categories = tf.concat([y_test], axis=0)
array = confusion_matrix(predicted_categories, true_categories, normalize='true')
df_cm = pd.DataFrame(array, index = [i for i in ['hornpipe', 'jig', 'polka', 'reel', 'slide']],
                  columns = [i for i in ['hornpipe', 'jig', 'polka', 'reel', 'slide']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

confusion matrix plotted for weight adjusted model

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 99.00%


In [ ]:
from keras.utils.vis_utils import plot_model
from PIL import Image
plot_model(model, to_file='multi_meter_model_diagram.png', show_shapes=True, show_layer_names=True)
display(Image.open('multi_meter_model_diagram.png'))